In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Reshape, Concatenate, Input, Lambda
from tensorflow import convert_to_tensor
from tensorflow.keras import backend
import numpy as np
backend.set_floatx('float32')

In [18]:
n1_1 = LSTM(10, return_sequences = True, name = 'n1_1')
n1_3 = LSTM(10, return_state = True, name = 'n1_3')
n1_d = Dense(5, kernel_initializer = 'glorot_uniform', activation = 'tanh', name = 'n1_d')

In [19]:
n2_1 = LSTM(10, return_sequences = True, name = 'n2_1')
n2_3 = LSTM(10, return_state = True, name = 'n2_3')
n2_d = Dense(5, kernel_initializer = 'glorot_uniform', activation = 'tanh', name = 'n2_d')

In [20]:
s_1 = LSTM(20, return_sequences = True, name = 's_1')
s_2 = LSTM(15, return_sequences = True, name = 's_2')
s_3 = LSTM(10, return_sequences = True, name = 's_3')
s_d = Dense(1, kernel_initializer = 'glorot_uniform', activation = 'tanh', name = 's_d')

In [21]:
def MODEL (Ts, Tn, Tg):
    # Tn: Number of words in each of Ts's news data
    # Ts: Number of stock prices. There are equal number of news data blocks (each containing Tn words)
    # Tg: Dimentionality of the GloVe vectors used
    
    N1 = Input(shape = (Ts, Tn, Tg), batch_size = 1, name = 'N1')
    N2 = Input(shape = (Ts, Tn, Tg), batch_size = 1, name = 'N2')
    S = Input(shape = (Ts), batch_size = 1, name = 'S')
    
    N_1 = Reshape((Ts, Tn, Tg))(N1)
    N_2 = Reshape((Ts, Tn, Tg))(N2)
    S_ = Reshape((Ts, 1))(S)

    sentiment = convert_to_tensor(np.zeros((1, 1, 10)))   #dim0 (batch_size) is super sticky...setting it to 1 (shape[0]) helps
    
    for i in range(Ts):
        var1 = Lambda(lambda N1: N1[:, i, :, :])(N_1)
        var2 = Lambda(lambda N2: N2[:, i, :, :])(N_2)
        
        var1 = Reshape((Tn, Tg))(var1)
        var2 = Reshape((Tn, Tg))(var2)
        
        var1 = n1_1(var1)
        var2 = n2_1(var2)
        
        _, _, var1 = n1_3(var1)
        _, _, var2 = n2_3(var2)
        
        var1 = n1_d(var1)
        var2 = n2_d(var2)
        
        var1 = Reshape((1, 5))(var1)
        var2 = Reshape((1, 5))(var2)
        
        var = Concatenate(axis = 2)([var1, var2])
        
        sentiment = Concatenate(axis= 1)([sentiment, var])
    
    
    sentiment = Lambda(lambda s: s[:, 1:, :])(sentiment)
    sentiment = Reshape((Ts, 10))(sentiment)
    S_ = Reshape((Ts, 1))(S_)
    
    X = Concatenate(axis = 2)([S_, sentiment])    #dims: (Ts, 51)
    
    X = s_1(X)
    X = s_2(X)
    X = s_3(X)
    X = s_d(X)
    
    model = Model(inputs = [N1, N2, S], outputs = X)
    
    return model

In [22]:
model = MODEL(2, 5, 7)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [23]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
N1 (InputLayer)                 [(1, 2, 5, 7)]       0                                            
__________________________________________________________________________________________________
N2 (InputLayer)                 [(1, 2, 5, 7)]       0                                            
__________________________________________________________________________________________________
reshape_46 (Reshape)            (1, 2, 5, 7)         0           N1[0][0]                         
__________________________________________________________________________________________________
reshape_47 (Reshape)            (1, 2, 5, 7)         0           N2[0][0]                         
____________________________________________________________________________________________

In [ ]:
model.compile(loss = 'mse', metrics = ['accuracy'])